<a href="https://colab.research.google.com/github/Daikikitazawa/python-study/blob/main/security_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Security Check Template
# 作成日：2025/05/06
# 更新日：2025/05/07
# 内容（古→新）：セキュリティチェックのテンプレ作成／Driveマウント処理の追加

# Google Driveへのマウント処理
import os
from google.colab import drive

def safe_mount_drive():
    try:
        from google.colab import drive
        if not os.path.ismount('/content/drive'):
            drive.mount('/content/drive')
            print("✅ Google Driveが正常にマウントされました。")
        else:
            print("✅ Google Driveは既にマウントされています。")
    except ImportError:
        print("⚠️ このコードはColab専用です。ローカル環境では不要です。")
safe_mount_drive()

# セキュリティチェックの実行
import json
import re
import os

def run_security_scan(notebook_path):
    """
    指定したJupyter Notebookファイル内のコード・出力に含まれる
    APIキーや個人情報を検出するためのセキュリティチェック関数。
    """
    danger_patterns = {
    "APIキー（OpenAI風）": r"sk-[a-zA-Z0-9]{20,}",
    "APIキー（一般）": r"[a-zA-Z0-9_\-]{32,}",
    "メールアドレス": r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+",
    "環境変数風": r"(AWS|SECRET|KEY|TOKEN|PASSWORD)[\w_]*\s*=\s*['\"].+['\"]",
    "認証ヘッダ": r"Authorization:\s*Bearer\s+[a-zA-Z0-9\-_\.=]+",
    "電話番号": r"(?:\+81|0)\d{1,4}[-\s]?\d{1,4}[-\s]?\d{4}",
    "OAuth系クレデンシャル": r"(client_id|client_secret|refresh_token)\s*[:=]\s*['\"].{10,}['\"]",
    "パスワード（平文）": r"password\s*[:=]\s*['\"].{6,}['\"]",
    "Slackトークン": r"xox[baprs]-[a-zA-Z0-9]{10,48}",
    "GitHubトークン": r"ghp_[A-Za-z0-9]{36}",
    "AWSアクセスキー": r"AKIA[0-9A-Z]{16}",
    "Slack Webhook URL": r"https://hooks\.slack\.com/services/[a-zA-Z0-9]+/[a-zA-Z0-9]+/[a-zA-Z0-9]+"
    }

    if not os.path.exists(notebook_path):
        print("❌ 指定したノートブックが見つかりません。パスを確認してください。")
        return

    with open(notebook_path, "r", encoding="utf-8") as f:
        nb = json.load(f)

    matches = []
    for cell in nb.get("cells", []):
        content_blocks = ["".join(cell.get("source", []))]
        for out in cell.get("outputs", []):
            if "text" in out:
                content_blocks.append("".join(out["text"]))
            if "data" in out and "text/plain" in out["data"]:
                content_blocks.append("".join(out["data"]["text/plain"]))

        for content in content_blocks:
            for label, pattern in danger_patterns.items():
                for match in re.finditer(pattern, content):
                    matches.append((label, match.group().strip()))

    if matches:
        print("🚨 リスクのある情報が検出されました。該当箇所を確認してください。：\n")
        for label, m in matches:
            print(f"[{label}] {m}")
    else:
        print("✅ セキュリティリスクは検出されませんでした。")

def scan_multiple_notebooks(notebook_paths):
    for path in notebook_paths:
        print(f"\n🔍 スキャン対象: {path}")
        run_security_scan(path)

# 実行例（使うときはパスを変更）
if __name__ == "__main__":
    notebooks = [
        "/content/drive/MyDrive/Colab Notebooks/python_test.ipynb",
        "/content/drive/MyDrive/Colab Notebooks/python_100practice.ipynb",
        "/content/drive/MyDrive/Colab Notebooks/security_check.ipynb"
    ]
    scan_multiple_notebooks(notebooks)